# Indice de ejecutabilidad de pliegos del GR y GL

#### Sustentado en:
    # Cartera disponible para los siguientes años
    # Tiempo en que se cerrarían los proyectos en ejecución
    # Ejecutabilidad en los últimos años
    # Ritmo para el cierre de cartera
    # 
    

#%% Información necesaria:
    
    # Información de ejecución anual en los últimos 7 años, de gasto en PROYECTOS, por tipología preferiblemente
    # Cartera disponible de proyectos por pliego

Primero importamos los módulos y paquetes

In [1]:
#Importar paquetes
import pandas as pd 
import numpy as np 
import os
from datetime import datetime, timedelta

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Definimos variables temporales

In [2]:
#Importamos variables temporales
now = datetime.now()
ayer = datetime.now() - timedelta(days=1)
anteayer = datetime.now() - timedelta(days=2)

# la fecha de corte es la más actual
fecha_corte = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y") # es la fecha actual en formato dd/mm/yyyy
print(now.strftime("%d") + '-' + now.strftime("%m") + '-' + now.strftime("%Y"))

15-04-2024


In [3]:
# Para limpiar variables duplicadas luego de un merge
def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

# Para crear el COD_ID de la ejecutora
def crear_id(df, NIVEL_GOB, SECTOR, PLIEGO, EJECUTORA):
    df = df.assign(COD_ID = np.where(df[NIVEL_GOB].str.startswith('2'), (df[PLIEGO].str.split('. ', n=1).str[0] +"-" + df[EJECUTORA].str.split('. ', n=1).str[0]),
                                     (df[SECTOR].str.split('. ', n=1).str[0] +"-" + df[PLIEGO].str.split('. ', n=1).str[0] +"-"+ df[EJECUTORA].str.split('. ', n=1).str[0])))
    return df

Se procede con la carga de información.
En principio es necesario contar con: 

1. Información presupuestaria del SIAF de segumiento de Gasto 2012-2022
2. Información presupuestaria del SIAF de segumiento de Gasto 2023-2024
3. Información de la cadena institucional (Sector, pliego, ejecutora y sec_ejec)
4. Base de PMI pretrabajada (SÍ requiere actualización diaria) - Banco de Inversiones
5. Consulta Avanzada de Datos abierto sobre el detalle de inversiones (datos del estado situacional de las inversiones, programación financiera, procesos de selección , entre otros.)

In [4]:
### Cargar bases 
# 1. Información presupuestaria del SIAF de segumiento de Gasto 2012-2022
concat_bd_consulta = pd.DataFrame()
for n in list(range(2012,2023)):
    bd = pd.read_csv(os.path.join(r'Y:\Financiamiento_Data\Data_CSV\SIAFGasto2012_2022', 'GastoConsolidado_' + str(n) + '.csv'), dtype={'CADENA_INSTITUCIONAL': str, 'PRODUCTO_PROYECTO': str, 'SEC_EJEC':str})
    bd['CODIGO_UNICO'] = bd['PRODUCTO_PROYECTO'].astype(int).astype(str)
    abc = bd[bd['CODIGO_UNICO']=='2318699']
    print(abc.shape, n)
    bd = bd[bd['CODIGO_UNICO'].str[0] == '2']
    bd = bd.groupby(['ANO_EJE', 'CADENA_INSTITUCIONAL', 'SEC_EJEC', 'CODIGO_UNICO']).agg({'TOTAL_DEVENGADO':'sum', 'PIA':'sum', 'PIM':'sum'}).reset_index()
    concat_bd_consulta = pd.concat([concat_bd_consulta, bd], ignore_index=True)  
    
# 2. Información presupuestaria del SIAF de segumiento de Gasto 2023-2024
bdg1 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2023', 'SeguimientoGasto_2023_04032024' + '.csv'), dtype={'COD_ID': str, 'COD_PRODUCTO_PROYECTO': str})
bdg2 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'SeguimientoGasto_2024_'+ fecha_corte +'.csv'), dtype={'COD_ID': str, 'COD_PRODUCTO_PROYECTO': str})

# 3. Información de la cadena institucional (Sector, pliego, ejecutora y sec_ejec)
bd_secejec = pd.read_excel(os.path.join(r'Y:\Data_CSV\Miscelanea', 'SEC_EJEC_UBIGEO' +".xlsx"), dtype={'SEC_EJEC': str})

# 4. Base de PMI pretrabajada (SÍ requiere actualización diaria) - Banco de Inversiones
bd_pmi = pd.read_csv(os.path.join(r'Y:\Data_CSV\BancoInv', 'InviertePe_'+fecha_corte+'.csv'), usecols=['CODIGO_UNICO', 'COSTO_ACTUAL_BCO', 'EXP_TCO', 'SITUACION', 'ESTADO', 'ORIGEN', 'ET_VIGENTE', 'DEV_ACUM_AL2023', 
            'FUNCION','DIVISION', 'GRUPO', 'TIPO_PROYECTO', 'CERRADO', 'POR_AVANCE_FISICO'] , dtype={'CODIGO_UNICO': str})

# 5. Consulta Avanzada de Datos abierto sobre el detalle de inversiones (datos del estado situacional de las inversiones, programación financiera, procesos de selección , entre otros.)
bd_consulta = pd.read_csv(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto11\Bases', 'DETALLE_INVERSIONES' + '.csv'), usecols=['CODIGO_UNICO', 'DEVEN_ACUMUL_AÑO_ANT', 'PRIMER_DEVENGADO', 'ULTIMO_DEVENGADO'], dtype={'CODIGO_UNICO': str})

(0, 29) 2012
(0, 29) 2013
(0, 29) 2014
(0, 29) 2015
(0, 29) 2016
(3, 29) 2017
(10, 29) 2018
(5, 29) 2019
(4, 29) 2020
(4, 29) 2021
(8, 29) 2022


In [5]:
concat_bd_consulta.info()
concat_bd_consulta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674462 entries, 0 to 674461
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ANO_EJE               674462 non-null  int64  
 1   CADENA_INSTITUCIONAL  674462 non-null  object 
 2   SEC_EJEC              674462 non-null  object 
 3   CODIGO_UNICO          674462 non-null  object 
 4   TOTAL_DEVENGADO       674462 non-null  float64
 5   PIA                   674462 non-null  float64
 6   PIM                   674462 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 36.0+ MB


,ANO_EJE,CADENA_INSTITUCIONAL,SEC_EJEC,CODIGO_UNICO,TOTAL_DEVENGADO,PIA,PIM
0,2012,E-01-001-012,1261,2030345,5221252.51,8504500.0,8562171.0
1,2012,E-01-001-012,1261,2160305,95933.30,0.0,136930.0
2,2012,E-01-018-001,1257,2094920,26841.72,0.0,26842.0
3,2012,E-01-018-001,1257,2094921,112310.40,128730.0,112311.0
4,2012,E-01-018-001,1257,2156676,10000.00,0.0,10000.0


In [6]:
concat_bd_consulta1=concat_bd_consulta.copy()
bd1 = bdg1.copy()
bd2 = bdg2.copy()
bd1['CODIGO_UNICO'] = bd1['COD_PRODUCTO_PROYECTO'].astype(int).astype(str)
bd2['CODIGO_UNICO'] = bd2['COD_PRODUCTO_PROYECTO'].astype(int).astype(str)
bd1['SEC_EJEC'] = bd1['SEC_EJEC'].astype(int).astype(str)
bd2['SEC_EJEC'] = bd2['SEC_EJEC'].astype(int).astype(str)
bd1['ANO_EJE'] = '2023'
bd2['ANO_EJE'] = '2024'
bd1['CADENA_INSTITUCIONAL'] = np.where(bd1['COD_ID'].str.len() == 10,
                                       'E'+'-'+ bd1['COD_ID'].astype(str),
                                       np.where(bd1['COD_ID'].str.len() == 7,
                                                'R'+'-'+ '99'+'-'+ bd1['COD_ID'].astype(str),
                                                np.where(bd1['COD_ID'].str.len()  == 8,
                                                         'M'+'-'+  bd1['COD_ID'].str.split('-',n=2).str[0] + bd1['COD_ID'].str.split('-',n=2).str[1] + bd1['COD_ID'].str.split('-',n=2).str[2],
                                                         bd1['COD_ID'])))

bd2['CADENA_INSTITUCIONAL'] = np.where(bd2['COD_ID'].str.len() == 10,
                                       'E'+'-'+ bd2['COD_ID'].astype(str),
                                       np.where(bd2['COD_ID'].str.len() == 7,
                                                'R'+'-'+ '99'+'-' + bd2['COD_ID'].astype(str),
                                                np.where(bd2['COD_ID'].str.len() == 8,
                                                         'M'+'-'+  bd2['COD_ID'].str.split('-',n=2).str[0] + bd2['COD_ID'].str.split('-',n=2).str[1] + bd2['COD_ID'].str.split('-',n=2).str[2],
                                                         bd2['COD_ID'])))
bd1 = bd1[bd1['CODIGO_UNICO'].str[0] == '2']
bd2 = bd2[bd2['CODIGO_UNICO'].str[0] == '2']
bd1 = bd1.groupby(['ANO_EJE', 'CADENA_INSTITUCIONAL', 'SEC_EJEC', 'CODIGO_UNICO']).agg({'TOTAL_DEVENGADO':'sum', 'PIA':'sum', 'PIM':'sum'}).reset_index()
bd2 = bd2.groupby(['ANO_EJE', 'CADENA_INSTITUCIONAL', 'SEC_EJEC', 'CODIGO_UNICO']).agg({'TOTAL_DEVENGADO':'sum', 'PIA':'sum', 'PIM':'sum'}).reset_index()
concat_bd_consulta2 = pd.concat([concat_bd_consulta1, bd1, bd2], ignore_index=True)
concat_bd_consulta2.info()
concat_bd_consulta2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788564 entries, 0 to 788563
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ANO_EJE               788564 non-null  object 
 1   CADENA_INSTITUCIONAL  788564 non-null  object 
 2   SEC_EJEC              788564 non-null  object 
 3   CODIGO_UNICO          788564 non-null  object 
 4   TOTAL_DEVENGADO       788564 non-null  float64
 5   PIA                   788564 non-null  float64
 6   PIM                   788564 non-null  float64
dtypes: float64(3), object(4)
memory usage: 42.1+ MB


,ANO_EJE,CADENA_INSTITUCIONAL,SEC_EJEC,CODIGO_UNICO,TOTAL_DEVENGADO,PIA,PIM
0,2012,E-01-001-012,1261,2030345,5221252.51,8504500.0,8562171.0
1,2012,E-01-001-012,1261,2160305,95933.30,0.0,136930.0
2,2012,E-01-018-001,1257,2094920,26841.72,0.0,26842.0
3,2012,E-01-018-001,1257,2094921,112310.40,128730.0,112311.0
4,2012,E-01-018-001,1257,2156676,10000.00,0.0,10000.0


In [7]:
secejec = bd_secejec.copy()
secejec['SEC_EJEC'] = secejec['SEC_EJEC'].astype(int).astype(str)
secejec['COD_ID'] = np.where((secejec['NIVEL_GOB'].str.split('. ').str[0]=='2'),
                             (secejec['PLIEGO'].str.split('. ', n=1).str[0] +"-" + secejec['EJECUTORA'].str.split('. ', n=1).str[0]),
                             (secejec['SECTOR'].str.split('. ', n=1).str[0] +"-" + secejec['PLIEGO'].str.split('. ', n=1).str[0] +"-"+
                              secejec['EJECUTORA'].str.split('. ', n=1).str[0]))

secejec['COD_PLIEGO'] = np.where(secejec['NIVEL_GOB'].str.split('. ').str[0]=='2', 
                                     secejec['SECTOR'].str.split('. ', n=1).str[0] + '-' + secejec['PLIEGO'].str.split('. ', n=1).str[0],
                                     secejec['SECTOR'].str.split('. ', n=1).str[0] + "-" + secejec['PLIEGO'].str.split('. ', n=1).str[0])
secejec['COD_PLIEGO'] = np.where(secejec['NIVEL_GOB'].str.split('. ').str[0]=='3', 
                                    secejec['SECTOR'].str.split('. ', n=1).str[0] + secejec['PLIEGO'].str.split('. ', n=1).str[0] + secejec['EJECUTORA'].str.split('. ', n=1).str[0],
                                    secejec['COD_PLIEGO'])
secejec['PLIEGO_EJEC'] = np.where(secejec['NIVEL_GOB'].str[0].isin({'1', '2'}), secejec['SECTOR'].str.split('. ', n=1).str[0]  + secejec['PLIEGO'],
                                     secejec['SECTOR'].str.split('. ', n=1).str[0] + secejec['PLIEGO'].str.split('. ', n=1).str[0] +
                                      secejec['EJECUTORA'].str.split('. ', n=1).str[0]+ '. ' +secejec['EJECUTORA'].str.split('. ', n=1).str[1])

secejec = secejec[['SEC_EJEC', 'COD_ID', 'COD_PLIEGO', 'PLIEGO_EJEC', 'NIVEL_GOB', 'SECTOR', 'PLIEGO', 'EJECUTORA']]
secejec_pl = secejec[['COD_PLIEGO', 'PLIEGO_EJEC', 'NIVEL_GOB', 'SECTOR', 'PLIEGO']].drop_duplicates(subset=['COD_PLIEGO'])
secejec_pl.info()
secejec_pl.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2106 entries, 0 to 2878
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   COD_PLIEGO   2106 non-null   object
 1   PLIEGO_EJEC  2106 non-null   object
 2   NIVEL_GOB    2106 non-null   object
 3   SECTOR       2106 non-null   object
 4   PLIEGO       2106 non-null   object
dtypes: object(5)
memory usage: 98.7+ KB


,COD_PLIEGO,PLIEGO_EJEC,NIVEL_GOB,SECTOR,PLIEGO
0,01-001,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS
5,01-002,01002. INSTITUTO NACIONAL DE ESTADISTICA E INF...,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,002. INSTITUTO NACIONAL DE ESTADISTICA E INFOR...
6,01-010,01010. DIRECCION NACIONAL DE INTELIGENCIA,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,010. DIRECCION NACIONAL DE INTELIGENCIA
7,01-011,01011. DESPACHO PRESIDENCIAL,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,011. DESPACHO PRESIDENCIAL
8,01-012,01012. COMISION NACIONAL PARA EL DESARROLLO Y ...,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,012. COMISION NACIONAL PARA EL DESARROLLO Y VI...


In [8]:
#%%
histcui12 = concat_bd_consulta2.copy()
histcui12.rename(columns={'TOTAL_DEVENGADO':'DEV_'}, inplace=True)
histcui12.rename(columns={'PIA':'PIA_'}, inplace=True)
histcui12.rename(columns={'PIM':'PIM_'}, inplace=True)
histcui12  = histcui12.pivot(index= ['CADENA_INSTITUCIONAL', 'CODIGO_UNICO', 'SEC_EJEC'], columns='ANO_EJE', values=['DEV_', 'PIA_', 'PIM_']).reset_index().rename_axis(None)
histcui12.columns = [f'{i}{j}' for i, j in histcui12.columns]
histcui12['DEV_12_24'] = histcui12[[i for i in histcui12.columns if i.startswith('DEV_')]].sum(axis=1)
histcui12 = histcui12[histcui12['DEV_12_24'].fillna(0)>0]
review = histcui12[histcui12['CADENA_INSTITUCIONAL']=='0']
review = histcui12[['CADENA_INSTITUCIONAL']].drop_duplicates()

# Creamos la variable del pliego
histcui12['COD_PLIEGO'] = np.where(histcui12['CADENA_INSTITUCIONAL'].str[0] == 'E', 
                               histcui12['CADENA_INSTITUCIONAL'].str.split('-',n=2).str[1] + '-' + histcui12['CADENA_INSTITUCIONAL'].str.split('-',n=3).str[2],
                               np.where(histcui12['CADENA_INSTITUCIONAL'].str[0] == 'R', histcui12['CADENA_INSTITUCIONAL'].str.split('-',n=2).str[1] + '-' + histcui12['CADENA_INSTITUCIONAL'].str.split('-',n=3).str[2],
                                        histcui12['CADENA_INSTITUCIONAL'].str.split('-',n=4).str[1])) 
histcui12.info()
histcui12.head()

<class 'pandas.core.frame.DataFrame'>
Index: 270659 entries, 1 to 336187
Data columns (total 44 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CADENA_INSTITUCIONAL  270659 non-null  object 
 1   CODIGO_UNICO          270659 non-null  object 
 2   SEC_EJEC              270659 non-null  object 
 3   DEV_2012              53351 non-null   float64
 4   DEV_2013              57939 non-null   float64
 5   DEV_2014              55477 non-null   float64
 6   DEV_2015              46644 non-null   float64
 7   DEV_2016              48521 non-null   float64
 8   DEV_2017              48997 non-null   float64
 9   DEV_2018              52311 non-null   float64
 10  DEV_2019              54423 non-null   float64
 11  DEV_2020              57038 non-null   float64
 12  DEV_2021              59958 non-null   float64
 13  DEV_2022              66404 non-null   float64
 14  DEV_2023              58504 non-null   float64
 15  DEV_2

,CADENA_INSTITUCIONAL,CODIGO_UNICO,SEC_EJEC,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,...,PIM_2017,PIM_2018,PIM_2019,PIM_2020,PIM_2021,PIM_2022,PIM_2023,PIM_2024,DEV_12_24,COD_PLIEGO
1,E-01-001-003,2194051,3,NaN,NaN,10904415.40,319272.52,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11223687.92,01-001
2,E-01-001-003,2194052,3,NaN,NaN,16106156.16,6421113.81,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527269.97,01-001
3,E-01-001-003,2235248,3,NaN,NaN,17469544.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17469544.00,01-001
4,E-01-001-003,2357130,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,1639270.0,NaN,NaN,NaN,1639270.00,01-001
6,E-01-001-003,2410948,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3739808.0,2833370.0,416912.0,260690.0,NaN,NaN,2384331.05,01-001


In [9]:
pmi = bd_pmi.copy()
pmi['CODIGO_UNICO'] = pmi['CODIGO_UNICO'].astype(int).astype(str)
pmi['DEV_ACUM_AL2023'] = pmi.groupby(['CODIGO_UNICO'])['DEV_ACUM_AL2023'].transform('max')
pmi = pmi.drop_duplicates(subset=['CODIGO_UNICO'])
pmi.info()
pmi.head()

<class 'pandas.core.frame.DataFrame'>
Index: 499314 entries, 0 to 516357
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CODIGO_UNICO       499314 non-null  object 
 1   TIPO_PROYECTO      499314 non-null  object 
 2   SITUACION          499314 non-null  object 
 3   ESTADO             499314 non-null  object 
 4   CERRADO            496603 non-null  object 
 5   EXP_TCO            453212 non-null  object 
 6   ET_VIGENTE         185812 non-null  object 
 7   COSTO_ACTUAL_BCO   499261 non-null  float64
 8   DEV_ACUM_AL2023    289607 non-null  float64
 9   FUNCION            466716 non-null  object 
 10  DIVISION           466713 non-null  object 
 11  GRUPO              466713 non-null  object 
 12  POR_AVANCE_FISICO  60224 non-null   float64
 13  ORIGEN             499314 non-null  object 
dtypes: float64(3), object(11)
memory usage: 57.1+ MB


,CODIGO_UNICO,TIPO_PROYECTO,SITUACION,ESTADO,CERRADO,EXP_TCO,ET_VIGENTE,COSTO_ACTUAL_BCO,DEV_ACUM_AL2023,FUNCION,DIVISION,GRUPO,POR_AVANCE_FISICO,ORIGEN
0,2280502,PI-SNIP,V,A,NO,SI,SI,15896362.25,15085876.82,TRANSPORTE,TRANSPORTE TERRESTRE,VÍAS VECINALES,94.58,SPMIE2
2,2318699,PI-SNIP,V,A,NO,SI,SI,26426259.05,7277438.20,AMBIENTE,"DESARROLLO ESTRATÉGICO, CONSERVACIÓN Y APROVEC...",GESTIÓN INTEGRADA DE LOS RECURSOS HÍDRICOS,28.58,E2
3,2429179,PI-INVIERTE,V,A,NO,NO,NaN,98946159.78,17500.00,TRANSPORTE,TRANSPORTE TERRESTRE,VÍAS VECINALES,NaN,SPMIE2
4,2430118,PI-INVIERTE,V,A,NO,SI,NO,33797778.23,17752913.09,AGROPECUARIA,RIEGO,INFRAESTRUCTURA DE RIEGO,43.45,SPMIE2
6,2478903,PI-INVIERTE,V,C,SI,SI,SI,1860869.70,1860869.70,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",GESTIÓN,INFRAESTRUCTURA Y EQUIPAMIENTO,53.18,E2


In [10]:
cavan = bd_consulta.copy()
cavan['CODIGO_UNICO'] = cavan['CODIGO_UNICO'].fillna(0).astype(int).astype(str)
cavan = cavan.drop_duplicates(subset=['CODIGO_UNICO'])
cavan['DEV_ACUM_ANIOPREV'] = cavan['DEVEN_ACUMUL_AÑO_ANT'].fillna(0)
cavan['MES_PRIMERDEV'] = cavan['PRIMER_DEVENGADO'].fillna(0).astype(int).astype(str) ## Fecha de Primer Devengado en STR
cavan['MES_ULTIMODEV'] = cavan['ULTIMO_DEVENGADO'].fillna(0).astype(int).astype(str) ## Fecha de Último Devengado en STR
cavan['PRIMER_DEV_ANO'] = cavan['MES_PRIMERDEV'].str[:4]
cavan['PRIMER_DEV_MES'] = cavan['MES_PRIMERDEV'].str[4:6]
cavan['ULTIMO_DEV_ANO'] = cavan['MES_ULTIMODEV'].str[:4]
cavan['ULTIMO_DEV_MES'] = cavan['MES_ULTIMODEV'].str[4:6]
cavan = cavan[['DEV_ACUM_ANIOPREV', 'CODIGO_UNICO', 'PRIMER_DEV_ANO', 'PRIMER_DEV_MES','ULTIMO_DEV_ANO', 'ULTIMO_DEV_MES']]
cavan.info()
cavan.head()

<class 'pandas.core.frame.DataFrame'>
Index: 249196 entries, 0 to 249262
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DEV_ACUM_ANIOPREV  249196 non-null  float64
 1   CODIGO_UNICO       249196 non-null  object 
 2   PRIMER_DEV_ANO     249196 non-null  object 
 3   PRIMER_DEV_MES     249196 non-null  object 
 4   ULTIMO_DEV_ANO     249196 non-null  object 
 5   ULTIMO_DEV_MES     249196 non-null  object 
dtypes: float64(1), object(5)
memory usage: 13.3+ MB


,DEV_ACUM_ANIOPREV,CODIGO_UNICO,PRIMER_DEV_ANO,PRIMER_DEV_MES,ULTIMO_DEV_ANO,ULTIMO_DEV_MES
0,20500.0,2590550,2023,05,2023,05
1,0.0,2530169,0,,0,
2,24650.0,2183872,2013,12,2014,09
3,0.0,2512413,0,,0,
4,10000.0,2547856,2023,12,2023,12


In [11]:
base = pd.merge(histcui12, pmi, on=['CODIGO_UNICO'], how='left', validate='m:1')
base = base.merge(secejec_pl[['COD_PLIEGO', 'PLIEGO_EJEC', 'NIVEL_GOB', 'SECTOR', 'PLIEGO']], on=['COD_PLIEGO'], how='left', validate='m:1')
base.info()
base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270659 entries, 0 to 270658
Data columns (total 61 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CADENA_INSTITUCIONAL  270659 non-null  object 
 1   CODIGO_UNICO          270659 non-null  object 
 2   SEC_EJEC              270659 non-null  object 
 3   DEV_2012              53351 non-null   float64
 4   DEV_2013              57939 non-null   float64
 5   DEV_2014              55477 non-null   float64
 6   DEV_2015              46644 non-null   float64
 7   DEV_2016              48521 non-null   float64
 8   DEV_2017              48997 non-null   float64
 9   DEV_2018              52311 non-null   float64
 10  DEV_2019              54423 non-null   float64
 11  DEV_2020              57038 non-null   float64
 12  DEV_2021              59958 non-null   float64
 13  DEV_2022              66404 non-null   float64
 14  DEV_2023              58504 non-null   float64
 15  

,CADENA_INSTITUCIONAL,CODIGO_UNICO,SEC_EJEC,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,...,DEV_ACUM_AL2023,FUNCION,DIVISION,GRUPO,POR_AVANCE_FISICO,ORIGEN,PLIEGO_EJEC,NIVEL_GOB,SECTOR,PLIEGO
0,E-01-001-003,2194051,3,NaN,NaN,10904415.40,319272.52,NaN,NaN,NaN,...,1.122369e+07,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",GESTIÓN,INFRAESTRUCTURA Y EQUIPAMIENTO,NaN,E3,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS
1,E-01-001-003,2194052,3,NaN,NaN,16106156.16,6421113.81,NaN,NaN,NaN,...,2.252727e+07,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",GESTIÓN,INFRAESTRUCTURA Y EQUIPAMIENTO,NaN,E3,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS
2,E-01-001-003,2235248,3,NaN,NaN,17469544.00,0.00,NaN,NaN,NaN,...,1.746954e+07,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",GESTIÓN,INFRAESTRUCTURA Y EQUIPAMIENTO,NaN,E3,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS
3,E-01-001-003,2357130,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.090465e+08,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",GESTIÓN,INFRAESTRUCTURA Y EQUIPAMIENTO,90.07,SPMIE2,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS
4,E-01-001-003,2410948,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.384331e+06,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",GESTIÓN,INFRAESTRUCTURA Y EQUIPAMIENTO,100.00,E3,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,1. GOBIERNO NACIONAL,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS


In [12]:
review = base[base['COD_PLIEGO'].isna()]
review.info()
review.head()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CADENA_INSTITUCIONAL  0 non-null      object 
 1   CODIGO_UNICO          0 non-null      object 
 2   SEC_EJEC              0 non-null      object 
 3   DEV_2012              0 non-null      float64
 4   DEV_2013              0 non-null      float64
 5   DEV_2014              0 non-null      float64
 6   DEV_2015              0 non-null      float64
 7   DEV_2016              0 non-null      float64
 8   DEV_2017              0 non-null      float64
 9   DEV_2018              0 non-null      float64
 10  DEV_2019              0 non-null      float64
 11  DEV_2020              0 non-null      float64
 12  DEV_2021              0 non-null      float64
 13  DEV_2022              0 non-null      float64
 14  DEV_2023              0 non-null      float64
 15  DEV_2024              0 non-null      fl

,CADENA_INSTITUCIONAL,CODIGO_UNICO,SEC_EJEC,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,...,DEV_ACUM_AL2023,FUNCION,DIVISION,GRUPO,POR_AVANCE_FISICO,ORIGEN,PLIEGO_EJEC,NIVEL_GOB,SECTOR,PLIEGO


In [13]:
# ORDENAMIENTO
base1=base.copy()
base1['TIPO_PROYECTO'] = np.where(base1['TIPO_PROYECTO'].isna(), 'FUERA DE INVIERTE', base1['TIPO_PROYECTO'])
base1 = base1[~(base1['TIPO_PROYECTO'].isin({'GENERICO', 'GESTION/PROGRAMA', 'FUERA DE INVIERTE', 'PROCOMPITE'}))]

#base1['CODIGO_UNICO2'] = np.where(base1['TIPO_PROYECTO']=='GENERICO', 'GENERICO', base1['CODIGO_UNICO'])
base1 = base1.groupby(['COD_PLIEGO', 'CODIGO_UNICO']).agg({'DEV_2012':'sum', 'DEV_2013':'sum', 'DEV_2014':'sum', 'DEV_2015':'sum', 'DEV_2016':'sum', 'DEV_2017':'sum', 
                                                           'DEV_2018':'sum', 'DEV_2019':'sum', 'DEV_2020':'sum', 'DEV_2021':'sum', 'DEV_2022':'sum',  'DEV_2023':'sum',
                                                           'DEV_2024':'sum','PIA_2012':'sum', 'PIA_2013':'sum', 'PIA_2014':'sum', 'PIA_2015':'sum', 'PIA_2016':'sum', 'PIA_2017':'sum', 
                                                           'PIA_2018':'sum', 'PIA_2019':'sum', 'PIA_2020':'sum', 'PIA_2021':'sum', 'PIA_2022':'sum',  'PIA_2023':'sum',
                                                           'PIA_2024':'sum', 'PIM_2012':'sum', 'PIM_2013':'sum', 'PIM_2014':'sum', 'PIM_2015':'sum', 'PIM_2016':'sum', 'PIM_2017':'sum', 
                                                           'PIM_2018':'sum', 'PIM_2019':'sum', 'PIM_2020':'sum', 'PIM_2021':'sum', 'PIM_2022':'sum',  'PIM_2023':'sum',
                                                           'PIM_2024':'sum','CERRADO':'first', 'DEV_ACUM_AL2023':'first', 'TIPO_PROYECTO':'first', 'SITUACION':'first', 
                                                           'ESTADO':'first', 'EXP_TCO':'first', 'FUNCION':'first', 'DIVISION':'first', 'GRUPO':'first', 'ET_VIGENTE':'first',
                                                           'COSTO_ACTUAL_BCO':'first', 'NIVEL_GOB':'first', 'PLIEGO_EJEC':'first', 'SECTOR':'first', 'PLIEGO':'first', 'POR_AVANCE_FISICO': 'first'}).reset_index()

base1['NRO_Ejecutores'] = base1.groupby(['CODIGO_UNICO'])['COD_PLIEGO'].transform('count')
base1 = base1.sort_values(by=['NRO_Ejecutores', 'CODIGO_UNICO'], ascending=[False, True])
base1 = base1[base1['NRO_Ejecutores']<=3]
base1 = base1[base1['COSTO_ACTUAL_BCO']>0]
base1.info()
base1.head()


<class 'pandas.core.frame.DataFrame'>
Index: 224431 entries, 22044 to 189303
Data columns (total 58 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   COD_PLIEGO         224431 non-null  object 
 1   CODIGO_UNICO       224431 non-null  object 
 2   DEV_2012           224431 non-null  float64
 3   DEV_2013           224431 non-null  float64
 4   DEV_2014           224431 non-null  float64
 5   DEV_2015           224431 non-null  float64
 6   DEV_2016           224431 non-null  float64
 7   DEV_2017           224431 non-null  float64
 8   DEV_2018           224431 non-null  float64
 9   DEV_2019           224431 non-null  float64
 10  DEV_2020           224431 non-null  float64
 11  DEV_2021           224431 non-null  float64
 12  DEV_2022           224431 non-null  float64
 13  DEV_2023           224431 non-null  float64
 14  DEV_2024           224431 non-null  float64
 15  PIA_2012           224431 non-null  float64
 16  PIA

,COD_PLIEGO,CODIGO_UNICO,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,DEV_2019,...,DIVISION,GRUPO,ET_VIGENTE,COSTO_ACTUAL_BCO,NIVEL_GOB,PLIEGO_EJEC,SECTOR,PLIEGO,POR_AVANCE_FISICO,NRO_Ejecutores
22044,022007,2023804,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,ADMINISTRACION,EDIFICACIONES PUBLICAS,SI,1505611.7,3. GOBIERNOS LOCALES,022007. MUNICIPALIDAD DISTRITAL DE SHUPLUY,02. ANCASH,20. YUNGAY,95.0,3
52647,060614,2023804,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,ADMINISTRACION,EDIFICACIONES PUBLICAS,SI,1505611.7,3. GOBIERNOS LOCALES,060614. MUNICIPALIDAD DISTRITAL DE SOCOTA,06. CAJAMARCA,06. CUTERVO,95.0,3
149323,150720,2023804,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,ADMINISTRACION,EDIFICACIONES PUBLICAS,SI,1505611.7,3. GOBIERNOS LOCALES,150720. MUNICIPALIDAD DISTRITAL DE SAN JUAN DE...,15. LIMA,07. HUAROCHIRI,95.0,3
64818,080701,2046091,0.00,223544.06,0.0,0.0,0.0,0.0,0.0,0.0,...,PROMOCION DE LA PRODUCCION AGRARIA,IRRIGACION,NO,7056091.0,3. GOBIERNOS LOCALES,080701. MUNICIPALIDAD PROVINCIAL DE CHUMBIVILC...,08. CUSCO,07. CHUMBIVILCAS,NaN,3
65741,080708,2046091,251983.68,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,PROMOCION DE LA PRODUCCION AGRARIA,IRRIGACION,NO,7056091.0,3. GOBIERNOS LOCALES,080708. MUNICIPALIDAD DISTRITAL DE VELILLE,08. CUSCO,07. CHUMBIVILCAS,NaN,3


In [14]:
# Porcentajes de avance

###########
base2 = base1.merge(cavan, on=['CODIGO_UNICO'], how='left', validate='m:1')

review = base2[base2['PRIMER_DEV_ANO'].isin({'0', np.nan})]

#review = base2[base2['TIPO_PROYECTO'].isna()]
#review = review.drop_duplicates(subset=['CODIGO_UNICO'])
#review = base2[base2['DEV_CUI_Prev12']<0]

#%%

# Filtros

# Nos quedamos con las inversiones activas o cerradas (no desactivadas)
base2 = base2[base2['ESTADO'].isin({'A', 'C'})]
#Añadir año de primer devengado para proyectos que no figuran el el detalle de inversiones de datos abierto y en el caso en que no tengan año del primer devengado
base2['PRIMER_DEV_ANO'] = base2['PRIMER_DEV_ANO'].replace({'0':np.nan})

base2['PRIMER_DEV_ANO']= np.where((base2['PRIMER_DEV_ANO'].isna())&
                                  (base2['DEV_2024'].fillna(0)!=0)&
                                  (base2['DEV_2024'].fillna(0)>0)&
                                  (base2['DEV_ACUM_AL2023'].fillna(0)<=0),
                                  '2024',
                                  np.where((base2['PRIMER_DEV_ANO'].isna())&
                                            (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                            (base2['DEV_2023'].fillna(0)!=0)&
                                            ((base2['DEV_2023'])<=(base2['DEV_ACUM_AL2023']+10))&
                                            ((base2['DEV_2023'])>=(base2['DEV_ACUM_AL2023']-10)), 
                                            '2023',
                                            np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                     (base2['DEV_2022'].fillna(0)!=0)&
                                                     (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                     ((base2[['DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                     ((base2[['DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                     '2022',
                                                      np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                               (base2['DEV_2021'].fillna(0)!=0)&
                                                               (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                               ((base2[['DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                               ((base2[['DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                               '2021',
                                                                np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                         (base2['DEV_2020'].fillna(0)!=0)&
                                                                         (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                         ((base2[['DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                         ((base2[['DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                                         '2020',
                                                                         np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                  (base2['DEV_2019'].fillna(0)!=0)&
                                                                                  (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                  ((base2[['DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                  ((base2[['DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                                                  '2019',
                                                                                  np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                           (base2['DEV_2018'].fillna(0)!=0)&
                                                                                           (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                           ((base2[['DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                           ((base2[['DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                                                           '2018',
                                                                                           np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                                    (base2['DEV_2017'].fillna(0)!=0)&
                                                                                                    (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                                    ((base2[['DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                                    ((base2[['DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                                                                    '2017',
                                                                                                    np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                                             (base2['DEV_2016'].fillna(0)!=0)&
                                                                                                             (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                                             ((base2[['DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                                             ((base2[['DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']-10)), 
                                                                                                             '2016',
                                                                                                             np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                                                      (base2['DEV_2015'].fillna(0)!=0)&
                                                                                                                      (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                                                      ((base2[['DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                                                      ((base2[['DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                                                                                      '2015',
                                                                                                                      np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                                                               (base2['DEV_2014'].fillna(0)!=0)&
                                                                                                                               (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                                                               ((base2[['DEV_2014', 'DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                                                               ((base2[['DEV_2014', 'DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)), 
                                                                                                                               '2014',
                                                                                                                               np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                                                                        (base2['DEV_2013'].fillna(0)!=0)&
                                                                                                                                        (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                                                                        ((base2[['DEV_2013', 'DEV_2014', 'DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                                                                        ((base2[['DEV_2013', 'DEV_2014', 'DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)),
                                                                                                                                        '2013',
                                                                                                                                        np.where((base2['PRIMER_DEV_ANO'].isna())&
                                                                                                                                                 (base2['DEV_2012'].fillna(0)!=0)&
                                                                                                                                                 (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                                                                                                                                 ((base2[['DEV_2012','DEV_2013', 'DEV_2014', 'DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))<=(base2['DEV_ACUM_AL2023']+10))&
                                                                                                                                                 ((base2[['DEV_2012','DEV_2013', 'DEV_2014', 'DEV_2015', 'DEV_2016','DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020','DEV_2021','DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']-10)),
                                                                                                                                                 '2012', base2['PRIMER_DEV_ANO'])))))))))))))





base2['PRIMER_DEV_ANO']= np.where((base2['PRIMER_DEV_ANO'].isna())&
                              #   (base2['DEV_2023'].fillna(0)!=0)&
                              #   (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                  (base2['DEV_2012'].fillna(0)==0)&
                                  (base2['DEV_2013'].fillna(0)==0)&
                                  (base2['DEV_2014'].fillna(0)==0)&
                                  (base2['DEV_2015'].fillna(0)==0)&
                                  (base2['DEV_2016'].fillna(0)==0)&
                                  (base2['DEV_2017'].fillna(0)==0)&
                                  (base2['DEV_2018'].fillna(0)==0)&
                                  (base2['DEV_2019'].fillna(0)==0)&
                                  (base2['DEV_2020'].fillna(0)==0)&
                                  (base2['DEV_2021'].fillna(0)==0)&
                                  (base2['DEV_2022'].fillna(0)==0)&
                                  ((base2['DEV_2023'])>=(base2['DEV_ACUM_AL2023'])),
                                  '2023',
                                  np.where((base2['PRIMER_DEV_ANO'].isna())&
                              #            (base2['DEV_2022'].fillna(0)!=0)&
                              #            (base2['DEV_ACUM_AL2023'].fillna(0)!=0)&
                                           (base2['DEV_2012'].fillna(0)==0)&
                                           (base2['DEV_2013'].fillna(0)==0)&
                                           (base2['DEV_2014'].fillna(0)==0)&
                                           (base2['DEV_2015'].fillna(0)==0)&
                                           (base2['DEV_2016'].fillna(0)==0)&
                                           (base2['DEV_2017'].fillna(0)==0)&
                                           (base2['DEV_2018'].fillna(0)==0)&
                                           (base2['DEV_2019'].fillna(0)==0)&
                                           (base2['DEV_2020'].fillna(0)==0)&
                                           (base2['DEV_2021'].fillna(0)==0)&
                                           ((base2[['DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023'])),
                                           '2022', base2['PRIMER_DEV_ANO']))
                                           
                               
#Quitamos los que empezaran ejecución antes del 2012
base2 = base2[base2['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', 
                                            '2019', '2020', '2021', '2022', '2023', '2024'})] # '2004', '2005', '2006', '2007', '2008',  '2009',  '2010', '2011',

# Proyectos con costo registrado
base2 = base2[base2['COSTO_ACTUAL_BCO'].fillna(0)>0]

base2 = base2[['COD_PLIEGO', 'CODIGO_UNICO', 'DEV_2012', 'DEV_2013', 'DEV_2014', 'DEV_2015', 'DEV_2016', 'DEV_2017', 'DEV_2018', 'DEV_2019', 'DEV_2020', 'DEV_2021', 'DEV_2022', 'DEV_2023', 'DEV_2024',
               'PIA_2012', 'PIA_2013', 'PIA_2014', 'PIA_2015', 'PIA_2016', 'PIA_2017', 'PIA_2018', 'PIA_2019', 'PIA_2020', 'PIA_2021', 'PIA_2022', 'PIA_2023', 'PIA_2024', 'PIM_2012', 'PIM_2013', 'PIM_2014', 
               'PIM_2015', 'PIM_2016', 'PIM_2017', 'PIM_2018', 'PIM_2019', 'PIM_2020', 'PIM_2021', 'PIM_2022',  'PIM_2023','PIM_2024', 'POR_AVANCE_FISICO', 'CERRADO', 'DEV_ACUM_AL2023', 'TIPO_PROYECTO', 
               'SITUACION', 'ESTADO', 'EXP_TCO', 'FUNCION', 'DIVISION', 'GRUPO', 'ET_VIGENTE', 'COSTO_ACTUAL_BCO', 'NIVEL_GOB', 'SECTOR', 'PLIEGO', 'PLIEGO_EJEC', 'NRO_Ejecutores', 'DEV_ACUM_ANIOPREV', 'PRIMER_DEV_ANO', 
               'PRIMER_DEV_MES', 'ULTIMO_DEV_ANO', 'ULTIMO_DEV_MES']]

lit1 = 1*(10**6) # 1 millon
lit2 = 5*(10**6) # 5 millones
lit3 = 10*(10**6) # 10 millones
lit4 = 20*(10**6) # 20 millones
lit5 = 30*(10**6) # 30 millones
lit6 = 50*(10**6) # 50 millones
lit7 = 100*(10**6) # 100 millones

base2['RANGO_COSTO'] = np.where(base2['COSTO_ACTUAL_BCO']<=lit1, '<1M',
                                np.where((base2['COSTO_ACTUAL_BCO']>lit1)&(base2['COSTO_ACTUAL_BCO']<=lit2), '1-5M',
                                         np.where((base2['COSTO_ACTUAL_BCO']>lit2)&(base2['COSTO_ACTUAL_BCO']<=lit3), '5-10M',
                                                  np.where((base2['COSTO_ACTUAL_BCO']>lit3)&(base2['COSTO_ACTUAL_BCO']<=lit4), '10-20M',
                                                           np.where((base2['COSTO_ACTUAL_BCO']>lit4)&(base2['COSTO_ACTUAL_BCO']<=lit5), '20-30M',
                                                                    np.where((base2['COSTO_ACTUAL_BCO']>lit5)&(base2['COSTO_ACTUAL_BCO']<=lit6), '30-50M',
                                                                             np.where((base2['COSTO_ACTUAL_BCO']>lit6)&(base2['COSTO_ACTUAL_BCO']<=lit7), '50-100M', 
                                                                                      np.where((base2['COSTO_ACTUAL_BCO']>lit7), '>100M',
                                                                                               ''))))))))

for yy in list(range(2012,2025)):
    base2['DEV_CUI_' + str(yy)] = base2.groupby(['CODIGO_UNICO'])['DEV_' + str(yy)].transform('sum') # esta variable es importante por los casos de coejecución  
base2['NRO_Ejecutores'] = base2.groupby(['CODIGO_UNICO'])['COD_PLIEGO'].transform('count') # con esta sabremos cuáles son las coejecutadas
base2['Coejecutado'] = np.where(base2['NRO_Ejecutores']==1, 'No', 'Si')
# luego de los anteriores datos recién podremos eliminar lo del GN para quedarnos solo GR y GL
#base_r = base_r[base_r['CADENA_INSTITUCIONAL'].str[0].isin({'M', 'R'})]
base2 = base2[~(base2['COD_PLIEGO'].str[0:3]=='097')] # quitamos mancomunidades  
base2 = base2[base2['NIVEL_GOB'].isin({'2. GOBIERNOS REGIONALES', '3. GOBIERNOS LOCALES'})]

#prueba2= base2[((base2['PRIMER_DEV_ANO'].isna())&(base2['DEV_2012'].fillna(0)==0)&(base2['DEV_2013'].fillna(0)==0)&(base2['DEV_2014'].fillna(0)==0)&(base2['DEV_2015'].fillna(0)==0)&(base2['DEV_2016'].fillna(0)==0)&(base2['DEV_2017'].fillna(0)==0)&(base2['DEV_2018'].fillna(0)==0)&(base2['DEV_2019'].fillna(0)==0)&(base2['DEV_2020'].fillna(0)==0)&
             #  (base2['DEV_2021'].fillna(0)==0)&(((base2[['DEV_2022', 'DEV_2023']].sum(axis=1))>=(base2['DEV_ACUM_AL2023']))|((base2['DEV_2023'])>=(base2['DEV_ACUM_AL2023']))))]

base2.info()
base2.head()   

<class 'pandas.core.frame.DataFrame'>
Index: 186109 entries, 23 to 224430
Data columns (total 78 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   COD_PLIEGO         186109 non-null  object 
 1   CODIGO_UNICO       186109 non-null  object 
 2   DEV_2012           186109 non-null  float64
 3   DEV_2013           186109 non-null  float64
 4   DEV_2014           186109 non-null  float64
 5   DEV_2015           186109 non-null  float64
 6   DEV_2016           186109 non-null  float64
 7   DEV_2017           186109 non-null  float64
 8   DEV_2018           186109 non-null  float64
 9   DEV_2019           186109 non-null  float64
 10  DEV_2020           186109 non-null  float64
 11  DEV_2021           186109 non-null  float64
 12  DEV_2022           186109 non-null  float64
 13  DEV_2023           186109 non-null  float64
 14  DEV_2024           186109 non-null  float64
 15  PIA_2012           186109 non-null  float64
 16  PIA_20

,COD_PLIEGO,CODIGO_UNICO,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,DEV_2019,...,DEV_CUI_2016,DEV_CUI_2017,DEV_CUI_2018,DEV_CUI_2019,DEV_CUI_2020,DEV_CUI_2021,DEV_CUI_2022,DEV_CUI_2023,DEV_CUI_2024,Coejecutado
23,99-451,2133720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,850309.59,973345.59,297489.18,77467.66,6451.79,Si
24,150901,2135518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33500.0,...,0.0,4000.0,0.00,33500.0,0.00,0.00,14000.00,2614199.30,1105541.68,Si
25,150905,2135518,0.0,0.0,0.0,0.0,0.0,4000.0,0.0,0.0,...,0.0,4000.0,0.00,33500.0,0.00,0.00,14000.00,2614199.30,1105541.68,Si
26,99-463,2135518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4000.0,0.00,33500.0,0.00,0.00,14000.00,2614199.30,1105541.68,Si
33,040702,2161852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,344240.8,301542.99,28266.0,55657.18,0.00,0.00,2254982.32,0.00,Si


## Indicadores para seguimiento avanzado de ejecución presupuestaria

In [27]:
#%% acercamiento de un perfil
#base_r = base1[base1['COD_PLIEGO']=='100102']
#base_r = base_r[base_r['CODIGO_UNICO']=='2113144']
base_r = base2.copy()
#--
# Nro de inversiones que alcanzaron ejecución por rangos +
# Periodo para alcanzar %s de ejecución - parte1
base_r['Alcanza_30%'] = '0'
base_r['Alcanza_70%'] = '0'
base_r['Alcanza_90%'] = '0'

for yy in list(range(2012,2025)):
    #base_r['DEV_CUI_' + str(yy)] = base_r.groupby(['CODIGO_UNICO'])['DEV_' + str(yy)].transform('sum')
    #print([i for i in base_r.columns if i.startswith('DEV_CUI_')])
    base_r['AX_DEV_CUI_'+str(yy)] = base_r['DEV_CUI_'+str(yy)]
    base_r['DEV_AlCorte'] = base_r[[i for i in base_r.columns if (i.startswith('AX_DEV_CUI_'))]].sum(axis=1)
    base_r['PORC_AVANCE_' + str(yy)] = base_r['DEV_AlCorte']/base_r['COSTO_ACTUAL_BCO'] # ya se verificó que sean proyectos con costo mayor a cero
    
   # base_r['Alcanza_30%'] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= 0.3)&(base_r['Alcanza_30%'] == '0'), str(yy), base_r['Alcanza_30%'])
   # base_r['Alcanza_70%'] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= 0.7)&(base_r['Alcanza_70%'] == '0'), str(yy), base_r['Alcanza_70%'])
   # base_r['Alcanza_90%'] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= 0.9)&(base_r['Alcanza_90%'] == '0'), str(yy), base_r['Alcanza_90%'])
    
    #print(base_r[base_r['CODIGO_UNICO']=='2159509']['DEV_AlCorte'].sum())
    #print(base_r[base_r['CODIGO_UNICO']=='2159509']['COSTO_ACTUAL_BCO'].sum())
    #print(base_r[base_r['CODIGO_UNICO']=='2159509']['PORC_AVANCE_' + str(yy)])
    
   # base_r['Inv_sup30PC_'+str(yy)] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= (0.3))&(base_r['PORC_AVANCE_' + str(yy)] < (0.7)), 1, 0)
   #  base_r['Inv_sup70PC_'+str(yy)] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= (0.7))&(base_r['PORC_AVANCE_' + str(yy)] < (0.9)), 1, 0)
   #  base_r['Inv_sup90PC_'+str(yy)] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= (0.9)), 1, 0) # &(base_r['PORC_AVANCE_' + str(yy)] < (0.7))
    
  #   base_r['Inv_sup30PC_'+str(yy)] = np.where((base_r['Alcanza_30%'] == str(yy)), 1, 0)
  #   base_r['Inv_sup70PC_'+str(yy)] = np.where((base_r['Alcanza_70%'] == str(yy)), 1, 0)
  #   base_r['Inv_sup90PC_'+str(yy)] = np.where((base_r['Alcanza_90%'] == str(yy)), 1, 0)
    
#-+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

#base_r['Inv_sup30PCT'] = ''
#base_r['Inv_sup70PCT'] = ''
#base_r['Inv_sup90PCT'] = ''
#for n in [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012]:
 #   base_r['Inv_sup30PCT'] = np.where(base_r['Inv_sup30PC_'+str(n)]==1, str(n), base_r['Inv_sup30PCT'])
  #  base_r['Inv_sup70PCT'] = np.where(base_r['Inv_sup70PC_'+str(n)]==1, str(n), base_r['Inv_sup70PCT'])
   # base_r['Inv_sup90PCT'] = np.where(base_r['Inv_sup90PC_'+str(n)]==1, str(n), base_r['Inv_sup90PCT'])

base_r.drop(columns=[i for i in base_r.columns if i.startswith('AX_DEV_CUI')], inplace=True)
#i.startswith('Inv_sup30PC_')|i.startswith('Inv_sup70PC_')|i.startswith('Inv_sup90PC_') <-----------------------------------------

# Revision del ratio de inicio y culminación de inversiones !

# Periodo para alcanzar %s de ejecución - parte2
 #for m in [30, 70, 90]:
 #    base_r['Alcanza_'+str(m)+'PC_A'] = base_r['Alcanza_'+str(m)+'%'].astype(str).astype(int) - base_r['PRIMER_DEV_ANO'].astype(str).astype(int)
  #   base_r['Alcanza_'+str(m)+'PC_A'] = np.where(base_r['Alcanza_'+str(m)+'PC_A']<0, np.nan, base_r['Alcanza_'+str(m)+'PC_A'])

# Iniciadas que se dejaron de ejecutar

# Interrupción de ejecución


    # 2. Periodos muerto de ejecución luego de que se iniciara a devengar, como máximo iniciado el 2021, 
base_r['Ultima_EjecConst'] = ''
base_r['Anos_Real_Ejec'] = 0
for yy in list(range(2012,2025)):
    base_r['Ultima_EjecConst'] = np.where(base_r['DEV_CUI_'+str(yy)]>0, str(yy),  base_r['Ultima_EjecConst'])
    base_r['Anos_Real_Ejec'] = np.where(base_r['DEV_CUI_'+str(yy)]>0, base_r['Anos_Real_Ejec'] + 1, base_r['Anos_Real_Ejec'])
    
base_r['Periodo_Ejecucion_IncFin'] = base_r['Ultima_EjecConst'].astype(int) - base_r['PRIMER_DEV_ANO'].astype(int) + 1 ## número potencial de años de ejecución
base_r['Tiempo_SinEjecut_Real'] = base_r['Periodo_Ejecucion_IncFin'] - base_r['Anos_Real_Ejec']

review = base_r[['CODIGO_UNICO', 'ULTIMO_DEV_ANO', 'Ultima_EjecConst']]

    # 3. Proyectos que iniciaron antes del 2021, no llegaron al 50% / 30% y se dejó de ejecutar hasta la fecha de corte 2024 (NO EJECUTARON 2024, no ejecutaron 2021 ni 2022)
base_r['Avnc_Menor10PC_OFF'] = np.where((base_r['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020'}))&(base_r['PORC_AVANCE_2024']<(0.1))&(base_r['DEV_CUI_2024'].fillna(0) == 0), 1, 0)
base_r['Avnc_10_30PC_OFF'] = np.where((base_r['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020'}))&(base_r['PORC_AVANCE_2024']>=(0.1))&(base_r['PORC_AVANCE_2024']<(0.3))
                                      &(base_r['DEV_CUI_2024'].fillna(0) == 0), 1, 0)
base_r['Avnc_30_50PC_OFF'] = np.where((base_r['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020'}))&(base_r['PORC_AVANCE_2023']>=(0.3))&(base_r['PORC_AVANCE_2024']<(0.5))
                                      &(base_r['DEV_CUI_2024'].fillna(0) == 0), 1, 0)

#base_r['Avnc_Menor10PC_OFF2'] = np.where((base_r['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020'}))&(base_r['PORC_AVANCE_2024']<(0.1))&(base_r['DEV_CUI_2024'].fillna(0) == 0)
                                        # &(base_r['DEV_CUI_2023'].fillna(0) == 0), 1, 0)
#base_r['Avnc_10_30PC_OFF2'] = np.where((base_r['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020'}))&(base_r['PORC_AVANCE_2024']>=(0.1))&(base_r['PORC_AVANCE_2024']<(0.3))
                                    #  &(base_r['DEV_CUI_2024'].fillna(0) == 0)&(base_r['DEV_CUI_2023'].fillna(0) == 0), 1, 0)
#base_r['Avnc_30_50PC_OFF2'] = np.where((base_r['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020'}))&(base_r['PORC_AVANCE_2024']>=(0.3))&(base_r['PORC_AVANCE_2024']<(0.5))
                                     # &(base_r['DEV_CUI_2024'].fillna(0) == 0)&(base_r['DEV_CUI_2023'].fillna(0) == 0), 1, 0)

    # 1. proyectos que iniciaron un año y luego no volvieron a devengar, y que no iniciaron como máximo el 2024, alcanzando como máximo un 30%
#base_r['EjecUnAno_ynomas'] = np.where(
 #                                     (base_r['PRIMER_DEV_ANO'] == base_r['Ultima_EjecConst'])&(base_r['PORC_AVANCE_2024']<=(0.3))&(base_r['PRIMER_DEV_ANO']!='2024'),
                                  #    1,
                                    #  0
                                  #    )

#base_r.drop(columns=[i for i in base_r.columns if i.startswith('PORC_AVANCE_')], inplace=True)

base_r.info()
base_r.head()


<class 'pandas.core.frame.DataFrame'>
Index: 186109 entries, 23 to 224430
Columns: 102 entries, COD_PLIEGO to Avnc_30_50PC_OFF
dtypes: float64(70), int32(4), int64(3), object(25)
memory usage: 143.4+ MB


,COD_PLIEGO,CODIGO_UNICO,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,DEV_2019,...,PORC_AVANCE_2022,PORC_AVANCE_2023,PORC_AVANCE_2024,Ultima_EjecConst,Anos_Real_Ejec,Periodo_Ejecucion_IncFin,Tiempo_SinEjecut_Real,Avnc_Menor10PC_OFF,Avnc_10_30PC_OFF,Avnc_30_50PC_OFF
23,99-451,2133720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.626877,0.638012,0.638939,2024,8,13,5,0,0,0
24,150901,2135518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33500.0,...,0.008815,0.456289,0.645525,2024,5,8,3,0,0,0
25,150905,2135518,0.0,0.0,0.0,0.0,0.0,4000.0,0.0,0.0,...,0.008815,0.456289,0.645525,2024,5,8,3,0,0,0
26,99-463,2135518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008815,0.456289,0.645525,2024,5,8,3,0,0,0
33,040702,2161852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007818,0.031977,0.031977,2023,5,8,3,1,0,0


In [28]:
#Indicadores Adicionales . #Análisis por pliego
base_i = base_r.copy()
#-+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
base_i = base_i.assign(Inv_Nuevas_Total = 0)
base_i = base_i.assign(Inv_sup30PC_Total = 0)
base_i = base_i.assign(Inv_sup70PC_Total = 0)
base_i = base_i.assign(Inv_sup90PC_Total = 0)
base_i = base_i.assign(EJE_Pliego_Total = 0)
base_i = base_i.assign(EJE_PromPC_Pliego_Total = 0)
base_i['Inv_Abiertas'] = 0
base_i['Inv_Ej90%Abiertas']= 0
base_i['Alcanza_30%'] = '0'
base_i['Alcanza_70%'] = '0'
base_i['Alcanza_90%'] = '0'
base_i['Anos_Real_Ejec'] = 0

# Inversiones abiertas antes de 2021 y que aún no están cerradas
for yy in [2018, 2019, 2020, 2021]:
    base_i['Inv_Abiertas'] = np.where((base_i['PRIMER_DEV_ANO'].isin({str(yy)}))&(base_i['ESTADO']=='A'), 1, base_i['Inv_Abiertas'])
base_i['Inv_Abiertas_Pliego'] = base_i.groupby(['COD_PLIEGO'])['Inv_Abiertas'].transform('sum') 

#Promedio anual de inversiones nuevas 
for yy in [2018, 2019, 2021, 2022, 2023]:
    base_i['Inv_Nuevas'+str(yy)] = np.where(base_i['PRIMER_DEV_ANO'].isin({str(yy)}), 1, 0)
    base_i['NRO_Inv_Nuevas_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['Inv_Nuevas'+str(yy)].transform('sum')
    base_i['Inv_Nuevas_Total'] = base_i[['Inv_Nuevas_Total', 'NRO_Inv_Nuevas_'+str(yy)]].sum(axis=1)
    base_i['Prom_Inv_Nuevas'] = ((base_i.groupby(['COD_PLIEGO'])['Inv_Nuevas_Total'].transform('max'))/5)

    # Promedio anual de inversiones que superan el 90% de ejecución
    base_i['Inv_sup90PC_'+str(yy)] = np.where((base_r['PORC_AVANCE_' + str(yy)] >= (0.9)), 1, 0) 
    base_i['NRO_Inv_sup90PC_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['Inv_sup90PC_'+str(yy)].transform('sum')
    base_i['Inv_sup90PC_Total'] = base_i[['Inv_sup90PC_Total','NRO_Inv_sup90PC_'+str(yy)]].sum(axis=1)
    base_i['Prom_Inv_sup90PC'] = ((base_i.groupby(['COD_PLIEGO'])['Inv_sup90PC_Total'].transform('max'))/5)
        
    # Promedio anual de inversiones que superan el 70% de ejecución
    base_i['Inv_sup70PC_'+str(yy)] = np.where((base_i['PORC_AVANCE_' + str(yy)] >= (0.7))&(base_i['PORC_AVANCE_' + str(yy)] < (0.9)), 1, 0)
    base_i['NRO_Inv_sup70PC_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['Inv_sup70PC_'+str(yy)].transform('sum')
    base_i['Inv_sup70PC_Total'] = base_i[['Inv_sup70PC_Total','NRO_Inv_sup70PC_'+str(yy)]].sum(axis=1)
    base_i['Prom_Inv_sup70PC'] = ((base_i.groupby(['COD_PLIEGO'])['Inv_sup70PC_Total'].transform('max'))/5) 
    
    # Promedio anual de inversiones que superan el 30% de ejecución
    base_i['Inv_sup30PC_'+str(yy)] = np.where((base_i['PORC_AVANCE_' + str(yy)] >= (0.3))&(base_i['PORC_AVANCE_' + str(yy)] < (0.7)), 1, 0)
    base_i['NRO_Inv_sup30PC_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['Inv_sup30PC_'+str(yy)].transform('sum')
    base_i['Inv_sup30PC_Total'] = base_i[['Inv_sup30PC_Total','NRO_Inv_sup30PC_'+str(yy)]].sum(axis=1)
    base_i['Prom_Inv_sup30PC'] = ((base_i.groupby(['COD_PLIEGO'])['Inv_sup30PC_Total'].transform('max'))/5)   

    # Promedio de años que llevan para alcanzar el 90% desde el inicio de ejecución presupuestaria
    base_i['Alcanza_90%'] = np.where((base_i['PORC_AVANCE_' + str(yy)] >= 0.9)&(base_i['Alcanza_90%'] == '0'), str(yy), base_i['Alcanza_90%'])
    base_i['Anos_Alcanza_90%'] = base_i['Alcanza_90%'].astype(str).astype(int) - base_i['PRIMER_DEV_ANO'].astype(str).astype(int)
    base_i['Anos_Alcanza_90%'] = np.where(base_i['Anos_Alcanza_90%']<0, np.nan, base_i['Anos_Alcanza_90%'])
    base_i['Prom_Anos_Alcanza_90%'] = (base_i.groupby(['COD_PLIEGO'])['Anos_Alcanza_90%'].transform('mean')) 
    
    # Promedio de años que llevan para alcanzar el 70% desde el inicio de ejecución presupuestaria
    base_i['Alcanza_70%'] = np.where((base_i['PORC_AVANCE_' + str(yy)] >= 0.7)&(base_i['PORC_AVANCE_' + str(yy)] < (0.9))&(base_i['Alcanza_70%'] == '0'), str(yy), base_i['Alcanza_70%'])
    base_i['Anos_Alcanza_70%'] = base_i['Alcanza_70%'].astype(str).astype(int) - base_i['PRIMER_DEV_ANO'].astype(str).astype(int)
    base_i['Anos_Alcanza_70%'] = np.where(base_i['Anos_Alcanza_70%']<0, np.nan, base_i['Anos_Alcanza_70%'])
    base_i['Prom_Anos_Alcanza_70%'] = (base_i.groupby(['COD_PLIEGO'])['Anos_Alcanza_70%'].transform('mean'))
    
    # Promedio de años que llevan para alcanzar el 30% desde el inicio de ejecución presupuestaria
    base_i['Alcanza_30%'] = np.where((base_i['PORC_AVANCE_' + str(yy)] >= 0.3)&(base_i['Alcanza_30%'] == '0'), str(yy), base_i['Alcanza_30%'])
    base_i['Anos_Alcanza_30%'] = base_i['Alcanza_30%'].astype(str).astype(int) - base_i['PRIMER_DEV_ANO'].astype(str).astype(int)
    base_i['Anos_Alcanza_30%'] = np.where(base_i['Anos_Alcanza_30%']<0, np.nan, base_i['Anos_Alcanza_30%'])
    base_i['Prom_Anos_Alcanza_30%'] = (base_i.groupby(['COD_PLIEGO'])['Anos_Alcanza_30%'].transform('mean'))
    
    # Inversiones con más del 90% de ejecución sin cerrar 
    base_i['Inv_Ej90%Abiertas'] = np.where((base_i['ESTADO']=='A')&(base_i['PORC_AVANCE_' + str(yy)] >= 0.9), 1, base_i['Inv_Ej90%Abiertas'])
    base_i['Inv_Ej90%Abiertas_Total'] = base_i.groupby(['COD_PLIEGO'])['Inv_Ej90%Abiertas'].transform('sum') 

    # Capacidad de ejecución promedio anual (soles)
    base_i['EJE_Pliego_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['DEV_'+str(yy)].transform('sum')
    base_i['EJE_Pliego_Total'] = base_i[['EJE_Pliego_Total', 'EJE_Pliego_'+str(yy)]].sum(axis=1)
    base_i['EJE_Prom_Pliego'] = ((base_i.groupby(['COD_PLIEGO'])['EJE_Pliego_Total'].transform('sum'))/5)
    
    # Capacidad de ejecución promedio anual (soles) (%)
    base_i['EJE_PromPC_Pliego_Total'] = base_i[['EJE_PromPC_Pliego_Total', 'PORC_AVANCE_'+str(yy)]].sum(axis=1)
    base_i['EJE_PromPC_Pliego'] = ((base_i.groupby(['COD_PLIEGO'])['EJE_PromPC_Pliego_Total'].transform('sum'))/5)  
    
    # Inversiones con un solo año de ejecución sin llegar a más del 30%
    base_i['Ejec1EJEC_ALC30%'] = np.where((base_i['PRIMER_DEV_ANO'] == base_i['Ultima_EjecConst'])&(base_i['PORC_AVANCE_2024']<=(0.3))&(base_i['PRIMER_DEV_ANO']!='2024'), 1, 0 )  
    base_i['Ejec1EJEC_ALC30%_Pliego'] = base_i.groupby(['COD_PLIEGO'])['Ejec1EJEC_ALC30%'].transform('sum')     
    
    # Promedio anual de años con ejecución real
    base_i['Anos_Real_Ejec'] = np.where(base_i['DEV_CUI_'+str(yy)]>0, base_i['Anos_Real_Ejec'] + 1, base_i['Anos_Real_Ejec'])
    base_i['Prom_Anos_conEJEC'] = (base_i.groupby(['COD_PLIEGO'])['Anos_Real_Ejec'].transform('mean'))

    # Promedio de interrupciones en la ejecución de una inversión
    base_i['Periodo_Ejecucion_IncFin'] = base_i['Ultima_EjecConst'].astype(int) - base_i['PRIMER_DEV_ANO'].astype(int) + 1 ## número potencial de años de ejecución
    base_i['Interrupcion_EJEC_INV'] = base_i['Periodo_Ejecucion_IncFin'] - base_i['Anos_Real_Ejec']
    base_i['Prom_Interrupcion_EJEC_INV'] = (base_i.groupby(['COD_PLIEGO'])['Interrupcion_EJEC_INV'].transform('mean'))

    ####revisar luego
    #Porcentaje de inversiones que inician con PIA y terminan con Ejecución 
    #Inversiones que inician con PIA y terminan con Ejecución 
    base_i['PIA_EJEC_'+str(yy)] = np.where((base_i['PIA_' + str(yy)] > 0)&(base_i['DEV_' + str(yy)] > 0), 1, 0)
    base_i['NRO_PIA_EJEC_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['PIA_EJEC_'+str(yy)].transform('sum') 
    #Inversiones que inician con PIA por año:
    base_i['Inv_APER_'+str(yy)] = np.where(base_i['PIA_' + str(yy)] > 0, 1, 0)
    base_i['NRO_Inv_APER_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['Inv_APER_'+str(yy)].transform('sum')
    
    base_i['%INV_PIA_EJEC_'+str(yy)] =  (base_i['NRO_PIA_EJEC_'+str(yy)]/base_i['NRO_Inv_APER_'+str(yy)])*100
    
    #Inversiones que se desfinancian anualmente (PIA>0, PIM=0)
    base_i['PIA_PIM0_'+str(yy)] = np.where((base_i['PIA_' + str(yy)] > 0)&(base_i['PIM_' + str(yy)] <= 0), 1, 0)
    base_i['NRO_PIA_PIM0_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['PIA_PIM0_'+str(yy)].transform('sum')

#Proyectos que se descontinúan al 10%
base_i['Descontinua_10'] = np.where((base_i['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020', '2021'}))&(base_i['PORC_AVANCE_2024']<=(0.1))&(base_i['DEV_CUI_2024'].fillna(0) == 0)
                                         &(base_i['DEV_CUI_2023'].fillna(0) == 0), 1, 0)
base_i['Descontinua_10%_Total'] = (base_i.groupby(['COD_PLIEGO'])['Descontinua_10'].transform('max'))

#Proyectos que se descontinúan al 30%
base_i['Descontinua_30'] = np.where((base_i['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020', '2021'}))&(base_i['PORC_AVANCE_2024']>(0.1))&(base_i['PORC_AVANCE_2024']<=(0.3))
                                      &(base_i['DEV_CUI_2024'].fillna(0) == 0)&(base_i['DEV_CUI_2023'].fillna(0) == 0), 1, 0)
base_i['Descontinua_30%_Total'] = (base_i.groupby(['COD_PLIEGO'])['Descontinua_30'].transform('max'))

#Proyectos que se descontinúan al 50%
base_i['Descontinua_50'] = np.where((base_i['PRIMER_DEV_ANO'].isin({'2012', '2013', '2014', '2015', '2016', '2017',  '2018', '2019', '2020', '2021'}))&(base_i['PORC_AVANCE_2024']>(0.3))&(base_i['PORC_AVANCE_2024']<=(0.5))
                                      &(base_i['DEV_CUI_2024'].fillna(0) == 0)&(base_i['DEV_CUI_2023'].fillna(0) == 0), 1, 0)
base_i['Descontinua_50%_Total'] = (base_i.groupby(['COD_PLIEGO'])['Descontinua_50'].transform('max'))

base_i.info()
base_i.head()

C:\Users\jchamba\AppData\Local\Temp\ipykernel_900\2126512215.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_i['PIA_PIM0_'+str(yy)] = np.where((base_i['PIA_' + str(yy)] > 0)&(base_i['PIM_' + str(yy)] <= 0), 1, 0)
C:\Users\jchamba\AppData\Local\Temp\ipykernel_900\2126512215.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_i['NRO_PIA_PIM0_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['PIA_PIM0_'+str(yy)].transform('sum')
C:\Users\jchamba\AppData\Local\Temp\ipykernel_900\2126512215.py:24: PerformanceWarning: Data

<class 'pandas.core.frame.DataFrame'>
Index: 186109 entries, 23 to 224430
Columns: 215 entries, COD_PLIEGO to Descontinua_50%_Total
dtypes: float64(96), int32(82), int64(12), object(25)
memory usage: 248.5+ MB


C:\Users\jchamba\AppData\Local\Temp\ipykernel_900\2126512215.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_i['NRO_PIA_EJEC_'+str(yy)] = base_i.groupby(['COD_PLIEGO'])['PIA_EJEC_'+str(yy)].transform('sum')
C:\Users\jchamba\AppData\Local\Temp\ipykernel_900\2126512215.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_i['Inv_APER_'+str(yy)] = np.where(base_i['PIA_' + str(yy)] > 0, 1, 0)
C:\Users\jchamba\AppData\Local\Temp\ipykernel_900\2126512215.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is

,COD_PLIEGO,CODIGO_UNICO,DEV_2012,DEV_2013,DEV_2014,DEV_2015,DEV_2016,DEV_2017,DEV_2018,DEV_2019,...,NRO_Inv_APER_2023,%INV_PIA_EJEC_2023,PIA_PIM0_2023,NRO_PIA_PIM0_2023,Descontinua_10,Descontinua_10%_Total,Descontinua_30,Descontinua_30%_Total,Descontinua_50,Descontinua_50%_Total
23,99-451,2133720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,59,81.355932,0,10,0,1,0,1,0,1
24,150901,2135518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33500.0,...,2,50.000000,0,0,0,1,0,1,0,1
25,150905,2135518,0.0,0.0,0.0,0.0,0.0,4000.0,0.0,0.0,...,0,NaN,0,0,0,1,0,1,0,0
26,99-463,2135518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,131,83.969466,0,12,0,1,0,1,0,1
33,040702,2161852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,100.000000,0,0,0,0,0,0,0,1


In [32]:
#%%
base_f= base_i.copy()
base_f['NIVEL_GOB'] = base_f['NIVEL_GOB'].replace({'2. GOBIERNOS REGIONALES':'GR', '3. GOBIERNOS LOCALES':'GL'})

base_f = base_f[['NIVEL_GOB', 'SECTOR', 'PLIEGO', 'PLIEGO_EJEC', 'COD_PLIEGO', 'CODIGO_UNICO', 'PIA_2018', 'PIA_2019', 'PIA_2020', 'PIA_2021', 'PIA_2022', 'PIA_2023', 'PIA_2024', 'PIM_2018', 'PIM_2019', 'PIM_2020', 'PIM_2021', 'PIM_2022',  'PIM_2023','PIM_2024', 
                 'DEV_2018', 'DEV_2019', 'DEV_2020', 'DEV_2021', 'DEV_2022', 'DEV_2023','DEV_2024', 'CERRADO', 'DEV_ACUM_AL2023', 'TIPO_PROYECTO', 'SITUACION', 'ESTADO', 'EXP_TCO', 'FUNCION', 'ET_VIGENTE', 'COSTO_ACTUAL_BCO', 'RANGO_COSTO',  'DIVISION', 'GRUPO',
                 'NRO_Ejecutores', 'Coejecutado', 'DEV_ACUM_ANIOPREV', 'PRIMER_DEV_ANO', 'ULTIMO_DEV_ANO', 'Ultima_EjecConst',  'POR_AVANCE_FISICO',
                 'Prom_Inv_Nuevas','Prom_Inv_sup30PC', 'Prom_Inv_sup70PC', 'Prom_Inv_sup90PC', 'Prom_Anos_Alcanza_30%', 'Prom_Anos_Alcanza_70%', 'Prom_Anos_Alcanza_90%', 'Ejec1EJEC_ALC30%', 'Prom_Anos_conEJEC',  'Prom_Interrupcion_EJEC_INV', 'Descontinua_10%_Total',
                 'Descontinua_30%_Total', 'Descontinua_50%_Total','Inv_Abiertas_Pliego','Inv_Ej90%Abiertas_Total', '%INV_PIA_EJEC_2018', '%INV_PIA_EJEC_2019', '%INV_PIA_EJEC_2021',  '%INV_PIA_EJEC_2022',  '%INV_PIA_EJEC_2023', 'NRO_PIA_PIM0_2018','NRO_PIA_PIM0_2019', 
                 'NRO_PIA_PIM0_2021','NRO_PIA_PIM0_2022','NRO_PIA_PIM0_2023', 'EJE_Prom_Pliego', 'EJE_PromPC_Pliego']]

base_f.rename(columns={'SECTOR':'Departamento', 'PLIEGO':'Provincia'}, inplace=True)

base_f['Departamento'] = np.where(base_f['NIVEL_GOB']=='GR', 'GRS', base_f['Departamento'] )
base_f['Provincia'] = np.where(base_f['NIVEL_GOB']=='GR', 'GRS', base_f['Provincia'] )
base_f.info()
base_f.head()

<class 'pandas.core.frame.DataFrame'>
Index: 186109 entries, 23 to 224430
Data columns (total 73 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   NIVEL_GOB                   186109 non-null  object 
 1   Departamento                186109 non-null  object 
 2   Provincia                   186109 non-null  object 
 3   PLIEGO_EJEC                 186109 non-null  object 
 4   COD_PLIEGO                  186109 non-null  object 
 5   CODIGO_UNICO                186109 non-null  object 
 6   PIA_2018                    186109 non-null  float64
 7   PIA_2019                    186109 non-null  float64
 8   PIA_2020                    186109 non-null  float64
 9   PIA_2021                    186109 non-null  float64
 10  PIA_2022                    186109 non-null  float64
 11  PIA_2023                    186109 non-null  float64
 12  PIA_2024                    186109 non-null  float64
 13  PIM_2018          

,NIVEL_GOB,Departamento,Provincia,PLIEGO_EJEC,COD_PLIEGO,CODIGO_UNICO,PIA_2018,PIA_2019,PIA_2020,PIA_2021,...,%INV_PIA_EJEC_2021,%INV_PIA_EJEC_2022,%INV_PIA_EJEC_2023,NRO_PIA_PIM0_2018,NRO_PIA_PIM0_2019,NRO_PIA_PIM0_2021,NRO_PIA_PIM0_2022,NRO_PIA_PIM0_2023,EJE_Prom_Pliego,EJE_PromPC_Pliego
23,GR,GRS,GRS,99451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE L...,99-451,2133720,0.0,0.0,0.0,0.0,...,94.444444,93.548387,81.355932,1,4,1,1,10,1.189020e+11,270.260854
24,GL,15. LIMA,09. OYON,150901. MUNICIPALIDAD PROVINCIAL DE OYON,150901,2135518,0.0,0.0,0.0,0.0,...,NaN,NaN,50.000000,0,0,0,0,0,7.823156e+08,89.529756
25,GL,15. LIMA,09. OYON,150905. MUNICIPALIDAD DISTRITAL DE NAVAN,150905,2135518,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,0,0,0,0,0,1.822646e+07,6.959670
26,GR,GRS,GRS,99463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,99-463,2135518,0.0,0.0,0.0,0.0,...,67.924528,75.294118,83.969466,4,6,15,16,12,1.523251e+11,456.377264
33,GL,04. AREQUIPA,07. ISLAY,040702. MUNICIPALIDAD DISTRITAL DE COCACHACRA,040702,2161852,0.0,0.0,0.0,0.0,...,100.000000,NaN,100.000000,0,0,0,0,0,1.365153e+07,3.532045


In [35]:
#%
outputFile = os.path.join(r'Y:\Capacidad de Inversión\Outputs', 'DataEjec_GRGL' +  fecha_corte + ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    base_f.to_excel(ew, sheet_name="DATA", index = False)        